In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage import io
from math import log10, sqrt
from PIL import Image

def PSNR(original,distored):

    original = cv2.resize(original, (512,512))
    distored = cv2.resize(distored, (512,512))

    mse_result = np.mean((original - distored) ** 2)
    max_pixel = 255.0
    psnr_result = 20 * log10(max_pixel / sqrt(mse_result))
    return psnr_result

def SSIM(original,distored):

    original = cv2.resize(original, (512,512))
    distored = cv2.resize(distored, (512,512))

    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    distored_gray = cv2.cvtColor(distored, cv2.COLOR_BGR2GRAY)

    score_result, diff = ssim(original_gray, distored_gray, full=True)
    diff_result = (diff * 255).astype("uint8")

    return score_result

if __name__ == "__main__":
    original = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Original/Original_origin/000134_0_0_0_0_0.jpg')
    distored = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Ours/Ours_original/5.Leg_50_down/000134_0_0_0_0_-50.png')
    PSNR_score = PSNR(original,distored)

    origin = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Original/Original_origin/000134_0_0_0_0_0.jpg')
    distored = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Ours/Ours_original/5.Leg_50_down/000134_0_0_0_0_-50.png')
    SSIM_score = SSIM(origin,distored)
    print('SSIM_score : %s'%SSIM_score)
    print('PSNR Score : %s'%PSNR_score)

In [ ]:
from PIL import ImageChops
import numpy as np
import cv2
from PIL import Image

def IDEA(original,distored,th=100):

    original = original.resize(original.size)
    distored = distored.resize(original.size)

    diff_od = ImageChops.difference(original, distored)

    diff_od = np.array(diff_od)
    diff_od = cv2.cvtColor(diff_od, cv2.COLOR_RGB2BGR)

    th = 100
    diff_od = np.where(diff_od <=th ,0,diff_od)
    diff_od = np.where(diff_od>th,255,diff_od)
    diff_od = np.where(diff_od==255,1,diff_od)
    score_od = np.sum(diff_od)/(original.size[0]*original.size[1])

    return score_od

def P2P(original,distored):

    h, w, c = original.shape
    gray_origin = cv2.resize(cv2.cvtColor(original, cv2.COLOR_BGR2GRAY), (w,h))
    gray_distored = cv2.resize(cv2.cvtColor(distored, cv2.COLOR_BGR2GRAY), (w,h))

    corners_origin = cv2.goodFeaturesToTrack(gray_origin, maxCorners=300, qualityLevel=0.3, minDistance=7)
    corners_distored, dist_status, dist_errors = cv2.calcOpticalFlowPyrLK(gray_origin, gray_distored, corners_origin, None)


    distances_distored = []
    for i in range(len(corners_origin)):
        x1, y1 = corners_origin[i].ravel()
        x2, y2 = corners_distored[i].ravel()
        distance_distored = ((x2-x1)**2 + (y2-y1)**2) ** 0.5
        distances_distored.append(distance_distored)

    # 이동 벡터의 크기 평균 계산
    if len(distances_distored) != 0:
        Distored_mean_distance = sum(distances_distored) / len(distances_distored)

    return Distored_mean_distance

if __name__ == "__main__":
    original = Image.open('/content/drive/MyDrive/192s2a/Final/Data/Original/Original_origin/000554_0_0_0_0_0.jpg')
    distored = Image.open('/content/drive/MyDrive/192s2a/Final/Data/Distored/Distored_origin/5.Leg_100_up/000554_0_0_0_0_100.jpg')
    IDEA_score = IDEA(original,distored)
    print('IDEA Score : %s'%IDEA_score)

    origin = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Original/Original_origin/000554_0_0_0_0_0.jpg')
    distored = cv2.imread('/content/drive/MyDrive/192s2a/Final/Data/Distored/Distored_origin/5.Leg_100_up/000554_0_0_0_0_100.jpg')
    P2P_score = P2P(origin,distored)
    print('P2P Score : %s'%P2P_score)